# Test multihead coherent state generation by pullback

Use the multihead (2-head) gates in the phase space 
to create a network that represent a coherent state,
by starting from a Gaussian state and making a pullback,
then propagating through a complex random medium

<img src="../img/coherentcomplexTikZ.png" width="900" height="210" />

<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>

nonlinearxwaves@gmail.com<br>
@created 22 july 2020<br>
@version 23 sep 2023<br>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable warning messages

In [2]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
tf_complex = tf.complex
tf_real = tf.float32
np_complex = complex
np_real = np.float64

## Dimension

In [4]:
N = 10

## Build vacuum by the Gaussian state

Build the layer

In [5]:
vacuum = ps.VacuumLayer(N)

## Build the pullback layer for generating the coherent state

Target displacement vector<br>
The displacement vector is a column vector

In [6]:
dinput = 3.0*np.ones((N,1))

Symplectic operator<br>
A simple identity matrix

In [7]:
D = ps.DisplacementLayerConstant(dinput)

## Define the layer for the complex medium
The LinearConstantMultiHead generate by default a non-trainable random medium with 
the relevant random symplectic operator

In [8]:
R = ps.RandomLayer(N)

## Connect the layers
Note the layers are in the inverse order as it is a pullback neural network

Input layers

In [9]:
xin = tf.keras.layers.Input(N)

Complex random medium

In [10]:
x2, a2 = R(xin)

Linear pullback for generating the coherent state

In [11]:
x1, a1 = D(x2,a2)

Gaussian vacuum state

In [12]:
chir, chii = vacuum(x1, a1)

Build the model

In [13]:
pullback = tf.keras.Model(inputs = xin, outputs=[chir, chii]) 

## Display the symplectic operator of the random layer

Evaluate the symplectic matrix and the inverse by using the builtin function

In [14]:
M, MI = R.get_M()

Print the symplectic matrix

In [15]:
utilities.printonscreen(M)

+0.5+0.0i -0.1+0.0i -0.1+0.0i +0.0+0.0i +0.5+0.0i +0.5+0.0i -0.1+0.0i -0.2+0.0i +0.4+0.0i +0.1+0.0i 
+0.1+0.0i +0.5+0.0i -0.0+0.0i -0.1+0.0i -0.5+0.0i +0.5+0.0i +0.2+0.0i -0.1+0.0i -0.1+0.0i +0.4+0.0i 
+0.5+0.0i -0.2+0.0i +0.4+0.0i -0.1+0.0i -0.1+0.0i -0.0+0.0i +0.2+0.0i +0.6+0.0i -0.1+0.0i +0.0+0.0i 
+0.2+0.0i +0.5+0.0i +0.1+0.0i +0.4+0.0i +0.0+0.0i -0.1+0.0i -0.6+0.0i +0.2+0.0i -0.0+0.0i -0.1+0.0i 
-0.3+0.0i +0.4+0.0i +0.5+0.0i -0.1+0.0i +0.6+0.0i +0.1+0.0i +0.3+0.0i +0.0+0.0i -0.2+0.0i -0.0+0.0i 
-0.4+0.0i -0.3+0.0i +0.1+0.0i +0.5+0.0i -0.1+0.0i +0.6+0.0i -0.0+0.0i +0.3+0.0i +0.0+0.0i -0.2+0.0i 
+0.3+0.0i +0.2+0.0i -0.1+0.0i +0.6+0.0i -0.0+0.0i -0.2+0.0i +0.6+0.0i -0.2+0.0i -0.0+0.0i -0.2+0.0i 
-0.2+0.0i +0.3+0.0i -0.6+0.0i -0.1+0.0i +0.2+0.0i -0.0+0.0i +0.2+0.0i +0.6+0.0i +0.2+0.0i -0.0+0.0i 
-0.2+0.0i +0.1+0.0i +0.3+0.0i +0.1+0.0i -0.1+0.0i -0.3+0.0i +0.1+0.0i +0.1+0.0i +0.7+0.0i +0.4+0.0i 
-0.1+0.0i -0.2+0.0i -0.1+0.0i +0.3+0.0i +0.3+0.0i -0.1+0.0i -0.1+0.0i +0.1+0.0i -0.4+0.0i +

In [16]:
tf.print(M)

[[0.468888164 -0.0584669709 -0.12911053 ... -0.151321545 0.439745665 0.105787225]
 [0.0584669709 0.468888164 -0.0112659149 ... -0.0562152788 -0.105787225 0.439745665]
 [0.547047615 -0.223233312 0.374205947 ... 0.648956716 -0.119094521 0.0367563851]
 ...
 [-0.239920408 0.273682982 -0.607466936 ... 0.614599645 0.178112969 -0.0364883393]
 [-0.193159699 0.0664976686 0.339802563 ... 0.0714701265 0.744518161 0.386051953]
 [-0.0664976686 -0.193159699 -0.147397503 ... 0.0824132338 -0.386051953 0.744518161]]


Print the inverse of the symplectic matrix

In [17]:
utilities.printonscreen(MI)

+0.5+0.0i +0.1+0.0i +0.5+0.0i +0.2+0.0i -0.3+0.0i -0.4+0.0i +0.3+0.0i -0.2+0.0i -0.2+0.0i -0.1+0.0i 
-0.1+0.0i +0.5+0.0i -0.2+0.0i +0.5+0.0i +0.4+0.0i -0.3+0.0i +0.2+0.0i +0.3+0.0i +0.1+0.0i -0.2+0.0i 
-0.1+0.0i -0.0+0.0i +0.4+0.0i +0.1+0.0i +0.5+0.0i +0.1+0.0i -0.1+0.0i -0.6+0.0i +0.3+0.0i -0.1+0.0i 
+0.0+0.0i -0.1+0.0i -0.1+0.0i +0.4+0.0i -0.1+0.0i +0.5+0.0i +0.6+0.0i -0.1+0.0i +0.1+0.0i +0.3+0.0i 
+0.5+0.0i -0.5+0.0i -0.1+0.0i +0.0+0.0i +0.6+0.0i -0.1+0.0i -0.0+0.0i +0.2+0.0i -0.1+0.0i +0.3+0.0i 
+0.5+0.0i +0.5+0.0i -0.0+0.0i -0.1+0.0i +0.1+0.0i +0.6+0.0i -0.2+0.0i -0.0+0.0i -0.3+0.0i -0.1+0.0i 
-0.1+0.0i +0.2+0.0i +0.2+0.0i -0.6+0.0i +0.3+0.0i -0.0+0.0i +0.6+0.0i +0.2+0.0i +0.1+0.0i -0.1+0.0i 
-0.2+0.0i -0.1+0.0i +0.6+0.0i +0.2+0.0i +0.0+0.0i +0.3+0.0i -0.2+0.0i +0.6+0.0i +0.1+0.0i +0.1+0.0i 
+0.4+0.0i -0.1+0.0i -0.1+0.0i -0.0+0.0i -0.2+0.0i +0.0+0.0i -0.0+0.0i +0.2+0.0i +0.7+0.0i -0.4+0.0i 
+0.1+0.0i +0.4+0.0i +0.0+0.0i -0.1+0.0i -0.0+0.0i -0.2+0.0i -0.2+0.0i -0.0+0.0i +0.4+0.0i +

Check if the matrix is symplectic by multipliying M and its inverse IM as generated by the layer

In [18]:
utilities.printonscreen(tf.matmul(M,MI))

+1.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i 
-0.0+0.0i +1.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i 
+0.0+0.0i -0.0+0.0i +1.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i -0.0+0.0i +1.0+0.0i +0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i 
-0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i +1.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i 
+0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +1.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i -0.0+0.0i -0.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i +0.0+0.0i 
-0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i 
-0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i -0.0+0.0i +0.0+0.0i +0.0+0.0i +

## Test the derivative of the model
Evaluate the expectation value of the displacement as derivative of the characterisc function

In [19]:
x = tf.Variable(np.zeros((1,N)), dtype=tf_real) # the derivative are evaluated at x=0
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    chir, chii = pullback(x)

Derivative of chir<br>
(must be zero)

In [20]:
utilities.printonscreen(tape.gradient(chir,x))

+0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i +0.0+0.0i 


In [21]:
tf.print(tape.gradient(chir,x))

[[0 0 0 ... 0 0 0]]


Derivative of chii<br>
(this is a random vector)

In [22]:
utilities.printonscreen(tape.gradient(chii,x))

+5.0+0.0i +2.4+0.0i +3.8+0.0i +1.8+0.0i +3.7+0.0i +1.4+0.0i +3.0+0.0i +1.2+0.0i +3.7+0.0i +1.5+0.0i 


In [23]:
tf.print(tape.gradient(chii,x))

[[4.97796202 2.39751244 3.81264353 ... 1.1661104 3.68405414 1.48972034]]


Test the conservation of the photon number by the norm of input and output displacement<br>
The total number of bosons for a coherent state is <br>
$\sum_{j=0}^{N-1} \frac{1}{2} d_j^2$

In [24]:
doutput=tape.gradient(chii,x).numpy(); print(doutput)

[[4.977962  2.3975124 3.8126435 1.829721  3.6850448 1.3814735 2.9913785
  1.1661104 3.6840541 1.4897203]]


In [25]:
print(np.linalg.norm(doutput)**2/2)

45.00000561509114


In [26]:
print(np.linalg.norm(dinput)**2/2)

45.0


Remark: The numbers are the same within numerical precision